In [1]:
import pandas as pd
handle_excel = pd.ExcelFile('20240126.xlsx')
agent = handle_excel.parse('WCC_Agent')
policy = handle_excel.parse('WCC_Policy')

<h4> Nhập kỳ CUTOFF <h4>

In [17]:
cutoff = '20240126'

In [2]:
agent_call = agent
policy_call = policy

In [26]:
agent_call = agent_call.fillna(value=0) #fillna để tránh lỗi trong việc đẩy data vào SQL
#3.Tạo Connection
conn_toSQL = pyo.connect(driver='{ODBC Driver 17 for SQL Server}', host = 'TEDPHO0134\TANDT_DP', database = 'DP_Tan', trusted_connection = 'yes',
                        ansi=True,autocommit=True)
cursor = conn_toSQL.cursor()

In [8]:
policy_call.columns.to_list()

['Period Cập nhật ',
 'Period Gỡ',
 'CONTRACT_NUMBER',
 'PO_NAME',
 'POLICY_STATUS',
 'AGENT_MOBILE',
 'SALE_OFFICE',
 'AGENT_CODE',
 'AGENT_NAME',
 'AGENT_GRADE',
 'ISSUE_DATE',
 'SALES UNIT',
 'DATE OF LATEST WCC',
 'FINAL WCC RESULT']

In [6]:
#Đổi tên cột
agent_call.rename(columns = {'Period Cập nhật ':"updatedPeriod",
 'Period Gỡ':"duePeriod",
 'WCC ROOKIES / WCC TRAINEE':"rookieTrainee",
 'Tháng gia nhập':"appointedDate",
 'Ngày thực hiện cuộc gọi gần nhất ':'recentCallDate',
 'KẾT QUẢ WCC':"Result"}, inplace = True) 

In [9]:
policy_call.rename(columns={
    'Period Cập nhật ': "updatedPeriod",
 'Period Gỡ': "duePeriod",
 'SALES UNIT': "SalesUnit",
 'DATE OF LATEST WCC': "LastedWCC",
 'FINAL WCC RESULT':"finalResult"
}, inplace = True)

In [11]:
policy_call.columns

Index(['updatedPeriod', 'duePeriod', 'CONTRACT_NUMBER', 'PO_NAME',
       'POLICY_STATUS', 'AGENT_MOBILE', 'SALE_OFFICE', 'AGENT_CODE',
       'AGENT_NAME', 'AGENT_GRADE', 'ISSUE_DATE', 'SalesUnit', 'LastedWCC',
       'finalResult'],
      dtype='object')

In [12]:
agent_call.columns

Index(['updatedPeriod', 'duePeriod', 'Agent_Number', 'Agent_Name',
       'rookieTrainee', 'appointedDate', 'recentCallDate', 'Result'],
      dtype='object')

<h4> Tạo cột CUTOFF <h4>

In [19]:
agent_call['CUTOFF'] = cutoff
policy_call['CUTOFF'] = cutoff

<h4> Tạo mới bảng trong SQL <h4>

In [20]:
#policy_call.to_sql("WCC_Policy", conn, if_exists='replace', index=False)

69

In [21]:
#agent_call.to_sql("WCC_Agent", conn, if_exists='replace', index=False)

189

In [31]:
agent_call = agent_call.fillna(value=0) #fillna để tránh lỗi trong việc đẩy data vào SQL
#Đẩy dữ liệu vào Bảng WCC_Agent
for index, row in agent_call.iterrows():
     cursor.execute("INSERT INTO dbo.WCC_Agent (updatedPeriod,duePeriod,Agent_Number,Agent_Name,rookieTrainee,appointedDate,recentCallDate,Result,CUTOFF) values(?,?,?,?,?,?,?,?,?)"
, 
row.updatedPeriod,
row.duePeriod,
row.Agent_Number,
row.Agent_Name,
row.rookieTrainee,
row.appointedDate,
row.recentCallDate,
row.Result,
row.CUTOFF

)
conn_toSQL.commit()

In [37]:
policy_call = policy_call.fillna(value=0)
#Đẩy dữ liệu vào Bảng WCC_Policy
for index, row in policy_call.iterrows():
     cursor.execute("INSERT INTO dbo.WCC_Policy (updatedPeriod,duePeriod,CONTRACT_NUMBER,PO_NAME,POLICY_STATUS,AGENT_MOBILE,SALE_OFFICE,AGENT_CODE,AGENT_NAME,AGENT_GRADE,ISSUE_DATE,SalesUnit,LastedWCC,finalResult,CUTOFF) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
, 
row.updatedPeriod,
row.duePeriod,
row.CONTRACT_NUMBER,
row.PO_NAME,
row.POLICY_STATUS,
row.AGENT_MOBILE,
row.SALE_OFFICE,
row.AGENT_CODE,
row.AGENT_NAME,
row.AGENT_GRADE,
row.ISSUE_DATE,
row.SalesUnit,
row.LastedWCC,
row.finalResult,
row.CUTOFF


)
conn_toSQL.commit()